# Zero shot using an API (LLAMA)

In [2]:
import pandas as pd
import os
from groq import Groq
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

Either notebook requires credentials to be loaded from a `.env` file, which should contain respectively either of the following lines, if not both:
```
GROQ_API_KEY=<your groq api key>
```

- [Groq API Key](https://console.groq.com/playground) can be generated and used free of charge

### Load df

In [3]:
df = pd.read_csv('../data/data_small.csv')

In [4]:
df.head()

,Unnamed: 0,dataset,text,logical_fallacies,source
0,18384,8,Testing on animals could save the life of you ...,appeal_to_emotion,NaN
1,11271,3,"I remember when China took over Hong Kong, I r...",none,NaN
2,15702,4,": The only ""Light at the End of the Tunnel"", i...",appeal_to_emotion,NaN
3,7148,3,So you only believe there are two ways to run ...,none,NaN
4,8147,3,Keep things the way they are or change them co...,false_dilemma,NaN


### Load dotenv with API Key

In [5]:
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [6]:
# validate API key
assert os.environ.get("GROQ_API_KEY"), "GROQ_API_KEY not found in .env file"

### Define LLM

In [7]:
import warnings
warnings.filterwarnings("ignore")
from langchain_groq import ChatGroq

#calling llm from platform, llama3 is called the llm
#temperature is hyperparameter, how creative do I want the llm to be (0 is not creative), sometimes, when it is not 0, it can give you the second likely word

llm = ChatGroq(
    model="llama3-8b-8192",  # Replace with your desired Llama model version
    temperature=0,           # No randomness, deterministic output
    max_tokens=None,         # Unlimited token length (adjust as needed)
    # timeout=None,            # No timeout (can be adjusted)
    max_retries=2            # Retry twice on transient failures
)

### Define a prompt

In [21]:
# Function to get a random text for each unique logical fallacy
def get_random_text_per_fallacy(df):
    result = {}
    unique_fallacies = df['logical_fallacies'].unique()
    for fallacy in unique_fallacies:
        random_text = df[df['logical_fallacies'] == fallacy].sample(n=1)['text'].iloc[0]
        result[fallacy] = random_text
    return result

# Get random text examples for each fallacy
random_texts = get_random_text_per_fallacy(df)

# Print the result
random_texts

{'appeal_to_emotion': "The most dangerous aspect of this proposal is, for the first time, we would delegate to computers the decision as to whether to start a war. That's dead wrong.",
 'none': "I don't think that kind of logic is good. It's essentially saying that so long as the authoritarians repress their people enough and nobody can rise up against them, we should think things are okay. We should strive to ensure they have leaders who value democracy, not oppression and authoritarianism.",
 'false_dilemma': 'I think that the Chinese government missed an opportunity here. Instead of controlling the narrative by having a limited press availability, they made it seem like they either having no press availability or to have to answer every little question asked. Why? Were they afraid for people to find out about what they talked about or was it that they did not want to antagonize any of the neighboring countries? Unfortunately, we will never really know. Most of the time you can tempe

In [22]:
def classify_fallacy(text: str) -> str:
    """Classifies text into one of the predefined logical fallacies."""
    try:
        # Combine system instructions and user content in one message
        # This avoids system message compatibility issues
        prompt = """Classify the following text into exactly one logical fallacy category:
- faulty_generalization
- ad_hominem
- false_dilemma 
- appeal_to_authority
- appeal_to_emotion  
- none

Here are definitions of each category for reference:
1. **Faulty Generalization**: This fallacy occurs when an argument assumes something is true for a large population without having a large enough sample. A kind of overgeneralization.
2. **Ad Hominem**: This fallacy occurs when the speaker is attacking the other person or some aspect of them rather than addressing the argument itself.
3. **False Dilemma**: This fallacy occurs when only two options are presented in an argument, even though more options may exist. A case of “either this or that”.
4. **Appeal to Authority**: This fallacy occurs when an argument relies on the opinion or endorsement of an authority figure who may not have relevant expertise or whose expertise is questionable. When applicable, a scientific consensus is not an appeal to authority.
5. **Appeal to Emotion**: This fallacy occurs when emotion is used to support an argument, such as pity, fear, anger, etc.
6. **None**: There are no fallacies in this text!

Here are examples of each category for reference:
1. **Faulty Generalization**: "I read one report about corruption, so that industry must be corrupt."
2. **Ad Hominem**: "Do you even know what you're talking about?"
3. **False Dilemma**: "Do you recommend drinking or injecting bleach to fight Covid?"
4. **Appeal to Authority**: "Trust me, I am a lawyer, so I know how to handle your taxes."
5. **Appeal to Emotion**: "You murdered 100,000 people, called Coronavirus a hoax, fired doctors, and told Americans to inject themselves with bleach. Maybe you should shut the fuck up."
6. **None**: "I don't think that kind of logic is good. It's essentially saying that so long as the authoritarians repress their people enough and nobody can rise up against them, we should think things are okay. We should strive to ensure they have leaders who value democracy, not oppression and authoritarianism."

Text to classify: {0}

Respond ONLY with the category name and nothing else.""".format(text)

        # Simplified message structure - only user message
        response = llm.invoke(prompt)
        
        # Properly extract content based on LangChain's response structure
        prediction = response.content.strip().lower()
        
        # Normalize the response
        valid_categories = ["faulty_generalization", "ad_hominem", "false_dilemma", 
                          "appeal_to_authority", "appeal_to_emotion", "none"]
        
        # Match to valid categories
        for category in valid_categories:
            if category in prediction:
                return category
                
        return prediction if prediction in valid_categories else "none"
        
    except Exception as e:
        print(f"Error processing text: {text[:50]}... | Error: {str(e)}")
        return "Error"

### Process dataframe

In [23]:
def process_dataframe(df: pd.DataFrame, batch_size=10) -> pd.DataFrame:
    """Process DataFrame with chunking for better performance."""
    result_df = df.copy()
    
    # Process in smaller batches to reduce API errors
    chunks = [df[i:i+batch_size] for i in range(0, len(df), batch_size)]
    
    with tqdm(total=len(df), desc="Classifying Logical Fallacies") as pbar:
        for chunk in chunks:
            # Process each text in the chunk
            chunk_results = []
            for text in chunk['text']:
                result = classify_fallacy(text)
                chunk_results.append(result)
                pbar.update(1)
                
            # Update results for this chunk
            result_df.loc[chunk.index, 'predicted_fallacy'] = chunk_results
    
    return result_df

### Make predictions

In [24]:
# make predictions only based on 1000 rows
df_small = df.iloc[:1000]

In [25]:
# Process the DataFrame and classify logical fallacies
processed_df = process_dataframe(df_small)

Classifying Logical Fallacies:  68%|██████▊   | 675/1000 [1:00:16<32:40,  6.03s/it]  

Error processing text: Violence should never be the answer.   This was cl... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499946, Requested 614. Please try again in 1m36.646s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: It's wild to me how you have given them the option... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499945, Requested 692. Please try again in 1m50.0704s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: ruin

Classifying Logical Fallacies:  68%|██████▊   | 678/1000 [1:00:16<17:39,  3.29s/it]

Error processing text: I feel that is a little too much information and i... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499944, Requested 557. Please try again in 1m26.5594s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: Since many of the students at St. Cloud State Univ... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499944, Requested 569. Please try again in 1m28.583s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: And 

Classifying Logical Fallacies:  68%|██████▊   | 680/1000 [1:00:16<12:21,  2.32s/it]

...

Error processing text: Artists  like MCZO and DJ duke lit the music scene... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499845, Requested 639. Please try again in 1m23.591s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: depends how you define growth. if any economic act... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499845, Requested 585. Please try again in 1m14.2078s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: It's

Classifying Logical Fallacies:  99%|█████████▉| 994/1000 [1:00:33<00:00, 20.86it/s]

Error processing text: This is dangerous. It isn't just naive; it's dange... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499844, Requested 555. Please try again in 1m8.832799999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: :  This is an EXPERIMENTAL vaccine. Just because y... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499844, Requested 592. Please try again in 1m15.1834s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text:

Classifying Logical Fallacies: 100%|██████████| 1000/1000 [1:00:34<00:00,  3.63s/it]

Error processing text: The internet can be a dangerous place for young ch... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499843, Requested 637. Please try again in 1m22.7894s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: While I don't agree with it, I do have points of c... | Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jpvzq9f4e11t7x9pmtnxzxe8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499842, Requested 674. Please try again in 1m29.137s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error processing text: : Is

### Evaluation

In [26]:
processed_df.head()

,Unnamed: 0,dataset,text,logical_fallacies,source,predicted_fallacy
0,18384,8,Testing on animals could save the life of you ...,appeal_to_emotion,NaN,faulty_generalization
1,11271,3,"I remember when China took over Hong Kong, I r...",none,NaN,none
2,15702,4,": The only ""Light at the End of the Tunnel"", i...",appeal_to_emotion,NaN,none
3,7148,3,So you only believe there are two ways to run ...,none,NaN,false_dilemma
4,8147,3,Keep things the way they are or change them co...,false_dilemma,NaN,false_dilemma


In [28]:
print(classification_report(processed_df["logical_fallacies"], processed_df["predicted_fallacy"]))

                       precision    recall  f1-score   support

                Error       0.00      0.00      0.00         0
           ad_hominem       0.62      0.39      0.48       101
  appeal_to_authority       0.46      0.30      0.37        56
    appeal_to_emotion       0.28      0.11      0.16       155
        false_dilemma       0.61      0.31      0.42        86
faulty_generalization       0.19      0.57      0.28       133
                 none       0.70      0.10      0.17       469

             accuracy                           0.22      1000
            macro avg       0.41      0.25      0.27      1000
         weighted avg       0.54      0.22      0.25      1000

